In [1]:
%load_ext autoreload
%autoreload 2

from gpu import set_gpu
import numpy as np
import os
import tensorflow as tf
from adapt.ml.dl_subclass import MLP, MLPprop, CNN, CNNprop, get_train, get_test
import adapt.utils.data_utils as prd
from adapt.ml.lda import train_lda, eval_lda

set_gpu()

Using TensorFlow backend.


Num GPUs Available:  1


In [2]:
path = 'C:/Users/yteh/Documents/work/necal/home data/TR58/DATA/MAT/'
all_files = os.listdir(path)
train_file = all_files[0]
adapt_file = all_files[1]
test_file = all_files[2]

In [3]:
train_data, train_params = prd.load_caps_train(path + train_file + '/traindata.mat')
train_data = train_data[:,:8,:]
train_dof = np.unique(train_params[:,2])
key = np.empty(train_dof.shape)
for i in range(len(train_dof)):
    key[i] = train_params[np.argmax(train_params[:,2] == train_dof[i]),0]

train_dof = train_dof[np.argsort(key)]
key = np.sort(key)

In [4]:
ep = 30
n_dof = np.max(train_params[:,0])
    
# Train NNs
mlp = MLP(n_class=n_dof)
cnn = CNN(n_class=n_dof)

optimizer = tf.keras.optimizers.Adam()
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.CategoricalAccuracy(name='train_accuracy')
train_prop_accuracy = tf.keras.metrics.MeanSquaredError(name='train_prop_accuracy')

trainmlp, traincnn, y_train, x_train_mlp, x_train_cnn, x_train_lda, y_train_lda, emg_scale, scaler, x_min, x_max, prop = prd.prep_train_caps(train_data, train_params, prop_b = False)

# Train neural networks
models = [mlp, cnn]
for model in models:
    if isinstance(model,CNN):
        ds = traincnn
    else:
        ds = trainmlp
    
    prop_b = isinstance(model, MLPprop) or isinstance(model, CNNprop)
    train_mod = get_train(prop = prop_b)

    for epoch in range(ep):
        # Reset the metrics at the start of the next epoch
        train_loss.reset_states()
        train_accuracy.reset_states()

        for x, y, y2 in ds:
            if prop_b:
                train_mod(x, y, model, optimizer, train_loss, train_accuracy, train_prop_accuracy, y2)
            else:
                train_mod(x, y, model, optimizer, train_loss, train_accuracy)

        if epoch == 0 or epoch == ep-1:
            print(
                f'Epoch {epoch + 1}, '
                f'Loss: {train_loss.result():.2f}, '
                f'Accuracy: {train_accuracy.result() * 100:.2f} '
            )
    del train_mod

# Train LDA
w,c, _, _, _ = train_lda(x_train_lda,y_train_lda)

mlp_w = mlp.get_weights()
cnn_w = cnn.get_weights()

Epoch 1, Loss: 1.78, Accuracy: 31.92 
Epoch 30, Loss: 0.35, Accuracy: 87.24 
Epoch 1, Loss: 1.19, Accuracy: 57.32 
Epoch 30, Loss: 0.11, Accuracy: 96.14 


In [12]:
acc = np.empty((len(all_files)-1,3))
train_dof = np.unique(train_params[:,2])

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.CategoricalAccuracy(name='test_accuracy')
for i in range(16,len(all_files)):
    test_loss.reset_states()
    test_accuracy.reset_states()

    test_file = all_files[i]
    test_data, test_params = prd.load_caps_train(path + test_file + '/traindata.mat')
    test_data = test_data[:,:8,:].astype('float64')
    test_dof = np.unique(test_params[:,2])

    test_key = np.empty(test_dof.shape)
    for dof_i in range(len(test_dof)):
        test_key[dof_i] = test_params[np.argmax(test_params[:,2] == test_dof[dof_i]),0]
    
    test_dof = test_dof[np.argsort(test_key)]
    test_key = np.sort(test_key)

    if not(np.all(np.in1d(test_dof,train_dof)) and np.all(np.in1d(train_dof,test_dof))):
        if len(test_dof) < len(train_dof):
            print('Missing classes')
            for key_i in key:
                test_params[test_params[:,2] == train_dof[int(key_i-1)],0] = key_i
        overlap = ~np.in1d(test_dof, train_dof)
        if overlap.any():
            print('Removing ' + test_dof[overlap])
            for ov_i in range(np.sum(overlap)):
                ind = test_params[:,2] == test_dof[overlap][ov_i]
                test_params[ind,:] = []
                test_data[ind,:] = []

    y_test, x_test_mlp, x_test_cnn, x_lda, y_lda = prd.prep_test_caps(test_data, test_params, scaler, emg_scale, num_classes=len(train_dof))

    # test CNN
    test_mod = get_test()
    test_mod(x_test_cnn, y_test, cnn, test_loss, test_accuracy)
    acc[i-1,0] = test_accuracy.result()*100

    # test MLP
    del test_mod
    test_loss.reset_states()
    test_accuracy.reset_states()

    test_mod = get_test()
    test_mod(x_test_mlp, y_test, mlp, test_loss, test_accuracy)
    acc[i-1,1] = test_accuracy.result()*100

    # test LDA
    acc[i-1,2] = eval_lda(w, c, x_lda, y_lda) * 100

    print ('Set: ' + test_file, f'CNN Accuracy: {acc[i-1,0]:.2f},', f'MLP Accuracy: {acc[i-1,1]:.2f},', f'LDA Accuracy: {acc[i-1,2]:.2f}')

Set: 20170808_104400 CNN Accuracy: 67.93, MLP Accuracy: 66.37, LDA Accuracy: 56.97
oopsie
Set: 20170913_053927 CNN Accuracy: 55.70, MLP Accuracy: 40.55, LDA Accuracy: 11.24
oopsie
Set: 20170913_061042 CNN Accuracy: 78.50, MLP Accuracy: 65.23, LDA Accuracy: 57.00
oopsie
Set: 20170915_173323 CNN Accuracy: 44.30, MLP Accuracy: 43.73, LDA Accuracy: 17.60
oopsie
Set: 20170916_161424 CNN Accuracy: 70.85, MLP Accuracy: 61.74, LDA Accuracy: 47.16
oopsie
Set: 20170923_073346 CNN Accuracy: 54.50, MLP Accuracy: 54.40, LDA Accuracy: 47.74
oopsie
Set: 20170929_051442 CNN Accuracy: 58.93, MLP Accuracy: 56.95, LDA Accuracy: 48.41


In [5]:
adapt_data, adapt_params = prd.load_caps_train(path + adapt_file + '/traindata.mat')
adapt_data = adapt_data[:,:8,:]
overlap = np.in1d(np.unique(adapt_params[:,2]), np.unique(train_params[:,2]))
overlap.all()

True

In [7]:
test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.CategoricalAccuracy(name='test_accuracy')

y_test, x_test_mlp, x_test_cnn, x_lda, y_lda = prd.prep_test_caps(adapt_data, adapt_params, scaler, emg_scale)

# test CNN
test_mod = get_test()
test_mod(x_test_cnn, y_test, cnn, test_loss, test_accuracy)
print (f'CNN Accuracy: {test_accuracy.result()*100:.2f}')

# test MLP
del test_mod
test_loss.reset_states()
test_accuracy.reset_states()

test_mod = get_test()
test_mod(x_test_mlp, y_test, mlp, test_loss, test_accuracy)
print (f'MLP Accuracy: {test_accuracy.result()*100:.2f}')

# test LDA
acc = eval_lda(w, c, x_lda, y_lda)
print (f'LDA Accuracy: {acc*100:.2f}')

CNN Accuracy: 36.87
MLP Accuracy: 69.24
LDA Accuracy: 62.54


In [10]:
ep = 5

trainmlp, traincnn, y_train, x_train_mlp, x_train_cnn, x_train_lda, y_train_lda, emg_scale, scaler, x_min, x_max, prop = prd.prep_train_caps(adapt_data, adapt_params, prop_b = False)

# Train neural networks
models = [mlp, cnn]
for model in models:
    if isinstance(model,CNN):
        ds = traincnn
    else:
        ds = trainmlp
    
    prop_b = isinstance(model, MLPprop) or isinstance(model, CNN)
    train_mod = get_train(prop = prop_b)

    for epoch in range(ep):
        # Reset the metrics at the start of the next epoch
        train_loss.reset_states()
        train_accuracy.reset_states()

        for x, y, y2 in ds:
            if prop_b:
                train_mod(x, y, model, optimizer, train_loss, train_accuracy, train_prop_accuracy, y2)
            else:
                train_mod(x, y, model, optimizer, train_loss, train_accuracy)

        if epoch == 0 or epoch == ep-1:
            print(
                f'Epoch {epoch + 1}, '
                f'Loss: {train_loss.result():.2f}, '
                f'Accuracy: {train_accuracy.result() * 100:.2f} '
            )
    del train_mod

# Train LDA
w,c, _, _, _ = train_lda(x_train_lda,y_train_lda)

mlp_w = mlp.get_weights()
cnn_w = cnn.get_weights()

Epoch 1, Loss: 0.68, Accuracy: 76.47 
Epoch 5, Loss: 0.37, Accuracy: 85.70 
Epoch 1, Loss: 0.61, Accuracy: 81.65 
Epoch 5, Loss: 0.21, Accuracy: 92.64 


In [8]:
test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.CategoricalAccuracy(name='test_accuracy')
test_loss.reset_states()
test_accuracy.reset_states()

test_data, test_params = prd.load_caps_train(path + test_file + '/traindata.mat')
test_data = test_data[:,:8,:].astype('float64')
y_test, x_test_mlp, x_test_cnn, x_lda, y_lda = prd.prep_test_caps(test_data, test_params, scaler, emg_scale)

# test CNN
test_mod = get_test()
test_mod(x_test_cnn, y_test, cnn, test_loss, test_accuracy)
print (f'CNN Accuracy: {test_accuracy.result()*100:.2f}')

# test MLP
del test_mod
test_loss.reset_states()
test_accuracy.reset_states()

test_mod = get_test()
test_mod(x_test_mlp, y_test, mlp, test_loss, test_accuracy)
print (f'MLP Accuracy: {test_accuracy.result()*100:.2f}')

# test LDA
acc = eval_lda(w, c, x_lda, y_lda)
print (f'LDA Accuracy: {acc*100:.2f}')

CNN Accuracy: 36.09
MLP Accuracy: 67.80
LDA Accuracy: 71.13
